In [22]:
import pandas as pd

import numpy as np

import datetime

from sklearn import linear_model

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn import metrics

from sklearn.preprocessing import PolynomialFeatures

from sklearn.preprocessing import StandardScaler

In [23]:
def comma_be_gone_into_int(x):
    return int(x.replace(',',''))

def BoolsInts(x):
    return np.array(list(map(int,x)))

In [24]:
df = pd.read_csv("kc_house_data_test_features.csv", index_col=[0])

df.reset_index(inplace = True)
df.drop('index', axis = 1,inplace = True)

In [25]:
df['yard'] = df.sqft_lot - (df.sqft_living/df.floors)

df['grade_3'] = [0 for i in range(len(df))]

In [26]:
new_column = []

for x in range(len(df)):
    new_column.append(df.iloc[x].date.split('T')[0][:4] + '-' + df.iloc[x].date.split('T')[0][4:6] + '-' + df.iloc[x].date.split('T')[0][6:])


df['date'] = [datetime.datetime.strptime(new_column[i], '%Y-%m-%d') for i in range(len(new_column))]


df['winter'] = np.array(list(map(int, [df['date'][i].month in [12,1,2] for i in range(len(df))])));
df['spring'] = np.array(list(map(int, [df['date'][i].month in [3,4,5] for i in range(len(df))])));
df['summer'] = np.array(list(map(int, [df['date'][i].month in [6,7,8] for i in range(len(df))])));
#df['autumn'] = np.array(list(map(int, [df['date'][i].month in [9,10,11] for i in range(len(df))])));
df.drop('date',axis=1, inplace = True)

---------------------------------------

In [27]:
len(df.columns)

24

---------------

In [28]:
df['renovated']= np.array(list(map(int,df.yr_renovated != 0)))

df.drop(['yr_renovated','yr_built'],axis=1, inplace = True)

In [29]:
zip_money_df = pd.read_csv('Seatle_Income_Population.csv')

zip_money_df['income'] = [comma_be_gone_into_int(x) for x in zip_money_df.income]

zip_money_df = zip_money_df[['zipcode','income']]


missing_zips_list = list(set(list(df.zipcode))-set(list(zip_money_df.zipcode)))


zip_money_dict = dict(zip(list(zip_money_df.zipcode),list(zip_money_df.income)))


for i in missing_zips_list:
    
    zip_money_dict[i] = sum(zip_money_df.income)/len(zip_money_df)

 
    
incomes_in_df_by_row_list = [[round(zip_money_dict[i])] for i in list(df.zipcode)]

df_income = pd.DataFrame.from_dict(dict(zip(list(df.index),incomes_in_df_by_row_list))).T

df_income.columns = ['income']

df['income'] = df_income 

In [30]:
len(df.columns)

24

In [31]:
mean_lat = df.lat.mean()

mean_long = df.long.mean()

df_lat_long_punnet = pd.concat([df.lat - mean_lat,df.long - mean_long],axis = 1)

df_lat_long_punnet['North_West'] = np.where((df_lat_long_punnet.lat > 0) & (df_lat_long_punnet.long > 0), True, False)

df_lat_long_punnet['North_East'] = np.where((df_lat_long_punnet.lat > 0) & (df_lat_long_punnet.long < 0), True, False) 

df_lat_long_punnet['South_West'] = np.where((df_lat_long_punnet.lat) < 0 & (df_lat_long_punnet.long > 0), True, False) 

df_lat_long_punnet['South_East'] = np.where((df_lat_long_punnet.lat < 0) & (df_lat_long_punnet.long < 0),True, False)  

df_lat_long_punnet.columns = ['lat', 'long','North_West','North_East','South_West','South_East']

df_ordinal = df_lat_long_punnet[['North_West','North_East','South_West','South_East']]

df_ordinal = df_ordinal.apply(BoolsInts)

df_ordinal.drop(['North_West'], axis = 1, inplace = True)

df = pd.concat([df,df_ordinal], axis = 1)

df.drop(['lat','long'],axis = 1, inplace = True)

In [32]:
len(df.columns)

25

In [33]:
categorical = ['view', 'condition', 'grade']

dummy_df = pd.get_dummies(data = df[categorical], columns = categorical, drop_first=True)

df = pd.concat([df,dummy_df], axis = 1)

df.drop(categorical,axis = 1, inplace = True)

In [34]:
len(df.columns)

40

In [35]:
df_features = df[df.drop('zipcode', axis = 1).columns[1:]]



poly_object = PolynomialFeatures(degree=2, include_bias=False)

poly = poly_object.fit_transform(df_features)

poly_columns = poly_object.get_feature_names(df_features.columns)

poly_df = pd.DataFrame(poly, columns = poly_columns)

df = pd.concat([df,poly_df],axis = 1)

In [36]:
len(df.columns)

819

In [37]:
dummy_zip_df = pd.get_dummies(data = df['zipcode'])
df = pd.concat([df,dummy_zip_df], axis = 1)
df.drop('zipcode',axis = 1, inplace = True)



In [38]:
len(dummy_zip_df.columns)

70

In [39]:
len(df.columns)

888

In [43]:
# Load

import pickle

with open('features.pickle', 'rb') as file:
    features = pickle.load(file)


In [46]:
with open('model.pickle', 'rb') as file:
    lr = pickle.load(file)

In [50]:
arr = lr.predict(df[features])

In [53]:
arr.tofile('housing_preds_jonathan_silverman.csv', sep = ',')

In [51]:
arr

array([489742.57978278, 510333.76860148, 450820.48536187, ...,
       322487.67593825, 427239.44018519, 314762.52486861])

In [44]:
len(features)

697

In [45]:
df[features]

,bedrooms,bedrooms,bathrooms,bathrooms,sqft_living,sqft_living,sqft_lot,sqft_lot,floors,floors,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,4,4.0,2.50,2.50,2270,2270.0,11500,11500.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,4,4.0,2.50,2.50,2270,2270.0,11500,11500.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,3,3.0,2.50,2.50,1470,1470.0,1779,1779.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,3,3.0,1.75,1.75,1280,1280.0,16200,16200.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,4.0,2.75,2.75,2830,2830.0,8126,8126.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,3,3.0,2.50,2.50,1530,1530.0,1131,1131.0,3.0,3.0,...,0,0,0,0,0,0,0,0,0,0
4318,4,4.0,2.50,2.50,2310,2310.0,5813,5813.0,2.0,2.0,...,1,0,0,0,0,0,0,0,0,0
4319,2,2.0,0.75,0.75,1020,1020.0,1350,1350.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
4320,3,3.0,2.50,2.50,1600,1600.0,2388,2388.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
